In [6]:
from pathlib import Path
from datetime import datetime, timedelta

def average_latency(sub_log: Path) -> float:
	readings = []
	with sub_log.open() as f:
		for line in f:
			if " - " in line:
				ts, val = line.split(" - ")
				readings.append((datetime.fromisoformat(ts), datetime.fromisoformat(val.strip())))

	# remove last value (is unusally large latency due to timeout)
	readings = readings[:-2]

	# only keep last 50 seconds to be safe
	last_ts = readings[-2][0]
	readings = [(ts, val) for ts, val in readings if ts > last_ts - timedelta(seconds=50)]

	# add 1s to values (cache flushing issue)
	readings = [(ts, val + timedelta(seconds=1))  for ts, val, in readings]


	latency = [(ts-val).total_seconds() for ts, val in readings]

	return sum(latency)/len(latency)

def average_cpu_usage(relay_perf: Path) -> float:
	readings = []
	with relay_perf.open() as f:
		for line in f:
			ts, val = line.split("   ")
			if val == "":
				raise ValueError("empty val")
			readings.append((datetime.fromisoformat(ts.strip()[:-1]), float(val.strip())))

	# only keep last 50 seconds to be safe
	last_ts = readings[-2][0]
	readings = [(ts, val) for ts, val in readings if ts > last_ts - timedelta(seconds=50)]

	percentages = [v for _, v in readings]

	return sum(percentages)/len(percentages)


def load_measurements(dir: Path):
	measurements = dir / Path("measurements")
	if not measurements.is_dir():
		raise ValueError(f"invalid directory: {measurements}")

	sub_logs = []
	relay_perf = []

	# organize files
	for file in measurements.iterdir():
		if file.name.startswith("clock"):
			sub_logs.append(file)
		if file.name.endswith("cpu_usage.log"):
			relay_perf.append(file)

	sub_avg_latencies = []
	for sub in sub_logs:
		try:
			sub_avg_latencies.append(average_latency(sub))
		except Exception as e:
			pass#print(f"{sub} skipped due to {e}")

	relay_perf_avg = {}
	for relay in relay_perf:
		name = relay.stem.replace("_cpu_usage", "")
		try:
			relay_perf_avg[name] = average_cpu_usage(relay)
		except Exception as e:
			print(f"{name} skipped due to {e}")


	return sub_avg_latencies, relay_perf_avg

load_measurements(Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement/star"))

([0.007216078431372548,
  0.013285921568627453,
  0.014453058823529414,
  0.007290901960784316,
  0.007843039215686276,
  0.00730549019607843,
  0.008280235294117649,
  0.013582450980392161,
  0.013459843137254903,
  0.014016117647058821,
  0.01329752941176471,
  0.013648333333333333,
  0.008475176470588236,
  0.013526156862745099,
  0.008515686274509804,
  0.008452921568627451,
  0.013211372549019612,
  0.014083686274509802,
  0.013180196078431374,
  0.01388862745098039,
  0.00823437254901961,
  0.014152960784313722,
  0.007168274509803922,
  0.014334431372549021,
  0.013537607843137255,
  0.014096941176470593,
  0.01357433333333333,
  0.007989333333333334,
  0.014309588235294117,
  0.013311137254901961,
  0.01379452941176471,
  0.014303784313725487,
  0.013641333333333332,
  0.013367960784313725,
  0.013572098039215687,
  0.014145627450980398,
  0.013382529411764706,
  0.008020764705882354,
  0.008415705882352941,
  0.008339294117647061,
  0.01395364705882353,
  0.014032235294117646,

In [7]:
from pprint import pprint
base = Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement")

measurements = []
for topo in ["full-mesh", "star", "leaf-and-spine", "two-relays", "single-relay"]:
	latencies, relays = load_measurements(base / topo)

	avg_lat = sum(latencies) / len(latencies)
	topo_results = {
		"name": topo,
		"average_latency": avg_lat,
		"relay_utilizations": relays
	}
	measurements.append(topo_results)
	pprint(topo_results)

{'average_latency': 0.013722714503205124,
 'name': 'full-mesh',
 'relay_utilizations': {'relay1': 0.4448979591836735,
                        'relay2': 0.4122448979591838,
                        'relay3': 0.41020408163265304,
                        'relay4': 0.41632653061224495,
                        'relay5': 0.36938775510204075}}
{'average_latency': 0.012437836997549016,
 'name': 'star',
 'relay_utilizations': {'hub': 0.2775510204081633,
                        'relay1': 0.4816326530612244,
                        'relay2': 0.5040816326530612,
                        'relay3': 0.4918367346938775,
                        'relay4': 0.4959183673469388}}
{'average_latency': 0.011827728509615403,
 'name': 'leaf-and-spine',
 'relay_utilizations': {'leaf1': 0.5879999999999999,
                        'leaf2': 0.566,
                        'leaf3': 0.6000000000000002,
                        'spine1': 0.13399999999999998,
                        'spine2': 0.06200000000000002}}
{'average

In [8]:
# scale CPU utilization to highest
highest_cpu = max(max(t["relay_utilizations"].values()) for t in measurements)

for t in measurements:
	t["average_latency"] = f"{t['average_latency'] * 1000} ms"
	for r, val in t["relay_utilizations"].items():
		t["relay_utilizations"][r] = f"{val / highest_cpu:.0%}"
pprint(measurements)

[{'average_latency': '13.722714503205124 ms',
  'name': 'full-mesh',
  'relay_utilizations': {'relay1': '25%',
                         'relay2': '23%',
                         'relay3': '23%',
                         'relay4': '23%',
                         'relay5': '20%'}},
 {'average_latency': '12.437836997549017 ms',
  'name': 'star',
  'relay_utilizations': {'hub': '15%',
                         'relay1': '27%',
                         'relay2': '28%',
                         'relay3': '27%',
                         'relay4': '27%'}},
 {'average_latency': '11.827728509615403 ms',
  'name': 'leaf-and-spine',
  'relay_utilizations': {'leaf1': '32%',
                         'leaf2': '31%',
                         'leaf3': '33%',
                         'spine1': '7%',
                         'spine2': '3%'}},
 {'average_latency': '16.28975929015838 ms',
  'name': 'two-relays',
  'relay_utilizations': {'relay1': '44%', 'relay2': '47%'}},
 {'average_latency': '17.3997634822